# Our First LLMAgent — Hailstone

In [1]:
LOGGING_ENABLED = True

In [2]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

## Define the Hailstone Tool

In [3]:
from pydantic import BaseModel

from llm_agents_from_scratch.tools import PydanticFunctionTool


class AlgoParams(BaseModel):
    """Params for next_number."""

    x: int


def next_number(params: AlgoParams) -> int:
    """Generate the next number of the sequence."""
    if params.x % 2 == 0:
        return params.x // 2
    return 3 * params.x + 1


# convert our Python function to a BaseTool
tool = PydanticFunctionTool(next_number)

## Define our backbone LLM

In [4]:
from llm_agents_from_scratch.llms import OllamaLLM

In [5]:
llm = OllamaLLM(model="qwen2.5:72b")

## Define the LLMAgent

In [6]:
from llm_agents_from_scratch import LLMAgent

In [7]:
llm_agent = LLMAgent(
    llm=llm,
    tools=[tool],
)

## Define the Hailstone Task

In [8]:
from llm_agents_from_scratch.data_structures import Task

In [9]:
instruction_template = """You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number x={x}.

<rules>
CALL `next_number` on the current number x
STOP AND WAIT for the result.
REPEAT this step-by-step process until the number 1 is reached.
FINAL RESULT: When you receive the number 1, provide the complete sequence
you observed from start to finish (including the starting number x and
ending number 1).
</rules>

<warnings>
NEVER fabricate or simulate tool call results
NEVER make multiple tool calls in one response
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
</warnings>
"""

In [10]:
task = Task(
    instruction=instruction_template.format(x=4),
)

In [11]:
handler = llm_agent.run(task)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: next_number
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: 2
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: The next number in the sequence is 2.

Sequence so far: 4, 2

<rules>
CALL `next_number` on the current number x=2
STOP AND WAIT for th...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: The assistant is following the instructions correctly. It has called the `next_number` function with the current number 2 and is waiting f...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙

In [14]:
handler.done()

True

# See the TaskResult

In [15]:
result = handler.result()

In [16]:
print(result)

**The final sequence observed from start to finish is: 4, 2, 1. The task has been successfully completed!**


### See the Rollout

In [17]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: The current instruction is 'You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number x=4.

<rules>
CALL `next_number` on the current number x
STOP AND WAIT for the result.
REPEAT this step-by-step process until the number 1 is reached.
FINAL RESULT: When you receive the number 1, provide the complete sequence
you observed from start to finish (including the starting number x and
ending number 1).
</rules>

<warnings>
NEVER fabricate or simulate tool call results
NEVER make multiple tool calls in one response
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
</warnings>
'

💬 assistant: I need to make the following tool call(s):

{
    "tool_name": "next_number",
    "arguments": {
        "x": 4
    }
}.

💬 tool: {
    "tool_call": {
        "tool_name": "next_number",
        "arguments": {
            "x": 4
        }
    },
    "content":

In [18]:
# number of task steps executed
handler.step_counter

3

## Evaluation

In [19]:
from dataclasses import dataclass


@dataclass
class Example:
    """Benchmark example."""

    number: int
    sequence: list[int]

    @property
    def min_steps(self) -> int:
        """Returns the minimum number of TaskSteps to perform the Task."""
        return len(self.sequence) - 1

In [20]:
benchmark: list[Example] = [
    Example(
        number=11,
        sequence=[11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(
        number=13,
        sequence=[13, 40, 20, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(
        number=4,
        sequence=[4, 2, 1],
    ),
    Example(
        number=3,
        sequence=[3, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(
        number=2,
        sequence=[2, 1],
    ),
]

### Run tasks

In [21]:
handlers = []
for ex in benchmark:
    h = llm_agent.run(
        Task(
            instruction=instruction_template.format(x=ex.number),
        ),
        max_steps=30,
    )
    handlers.append(h)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number

In [38]:
[h.done() for h in handlers]

[True, True, True, True, True]

In [39]:
[h.result() for h in handlers]

[TaskResult(task_id='cb1227c7-7d70-4d88-9de7-8d9c6d7d5dd1', content='The complete sequence from 17 to 1 is: 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1'),
 TaskResult(task_id='92788cb3-b776-41e0-a342-8640c5e84c28', content='The assistant has correctly followed the instructions and generated the sequence as required. The final result is accurate and complete: [13, 40, 20, 10, 5, 16, 8, 4, 2, 1]. No further action is needed.'),
 TaskResult(task_id='abc812df-95f8-4ca5-9040-45641bf741f0', content='The assistant has correctly followed the instructions and provided the final result as requested. The complete sequence from start to finish is [4, 2, 1], which includes the starting number 4 and the ending number 1.'),
 TaskResult(task_id='5b1e0f74-0d75-416b-ae5e-c25acc44bc52', content='The assistant has successfully followed the instructions and reached the number 1. The final sequence is correctly provided as: **3, 10, 5, 16, 8, 4, 2, 1**. No further action is required.'),
 TaskResult(task_i

## Result Correctness Evaluation

### Setup Judge LLM

In [40]:
judge_prompt_template = """You are an evaluation assistant. Given a number and its correct
sequence, use them to assess whether another assistants final result contains the correct sequence.

<number>
{number}
</number>

<sequence>
{sequence}
</sequence>

<result>
{result}
</result>
"""

#### Structured outputs for final result correctness evaluation

In [41]:
from pydantic import Field


class ExampleResultEvaluation(BaseModel):
    """Evaluation of result."""

    correct: bool = Field(
        description="True if the assistant's final result contains the correct sequence. False otherwise.",
    )

In [42]:
import asyncio

eval_async_tasks = []
for ex, handler in zip(benchmark, handlers, strict=False):
    async_task = llm.structured_output(
        prompt=judge_prompt_template.format(
            number=ex.number,
            sequence=ex.sequence,
            result=str(handler.result()),
        ),
        mdl=ExampleResultEvaluation,
    )
    eval_async_tasks.append(async_task)

result_evals = await asyncio.gather(*eval_async_tasks)
result_evals

[ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=True)]

## Number Steps Evaluation

In [43]:
num_steps_diff = [
    (h.step_counter - ex.min_steps)
    for h, ex in zip(handlers, benchmark, strict=False)
]
num_steps_diff

[15, 10, 1, 4, 1]

## LLM Agent Trajectory Evaluation

### LLM Judge Setup

In [55]:
llm_trajectory_judge = OllamaLLM(model="qwen2.5:3b")

In [44]:
trajectory_judge_prompt_template = """You are an evaluation assistant for evaluating another
assistant's reasoning trajectory for solving a sequence generation task. The instruction and correct
sequence is provided in addition to the trajectory of the assistant. Use the provided rubric
to provide an evaluation of the trajectory.

INSTRUCTION:
{instruction}

SEQUENCE:
{sequence}

TRAJECTORY:
{trajectory}

RUBRIC:
Reasoning (0-5): Does the assistant follow logical steps and make sound decisions?
Process (0-5): Does the assistant follow proper tool usage and protocols?

Provide a score, with 0 being lowest and 5 the highest, for each dimension and brief justification.

WARNINGS:
DO NOT DEDUCT FOR THE SAME ERROR MORE THAN ONCE. If an issue affects multiple dimensions, choose the most relevant dimension to dock points.
"""

In [45]:
from typing import Literal


class DimensionScore(BaseModel):
    """Model for a dimension score."""

    score: Literal[0, 1, 2, 3, 4, 5] = Field(
        description="Overall score on the dimension.",
    )
    justification: str = Field(
        description="Justification for the score.",
    )


class ExampleTrajectoryEvaluation(BaseModel):
    """Evaluation of LLM agent trajectory."""

    reasoning: DimensionScore
    process: DimensionScore

In [60]:
eval_async_tasks = []
for ex, handler in zip(benchmark, handlers, strict=False):
    async_task = llm_trajectory_judge.structured_output(
        prompt=trajectory_judge_prompt_template.format(
            instruction=instruction_template.format(x=ex.number),
            sequence=ex.sequence,
            trajectory=handler.rollout,
        ),
        mdl=ExampleTrajectoryEvaluation,
    )
    eval_async_tasks.append(async_task)

trajectory_evals = await asyncio.gather(*eval_async_tasks)
trajectory_evals

[ExampleTrajectoryEvaluation(reasoning=DimensionScore(score=4, justification='The assistant follows logical steps in solving the problem and making decisions based on the sequence generated by the `next_number` tool. However, there was a minor oversight where the assistant should have checked if the final number (1) completes the sequence before providing it as part of the result.'), process=DimensionScore(score=4, justification='The assistant uses tools appropriately and follows the protocol for checking and confirming the completion of the sequence. However, there is a minor oversight in reporting the complete sequence.')),
 ExampleTrajectoryEvaluation(reasoning=DimensionScore(score=4, justification="The assistant follows a logical step-by-step process in calling the `next_number` tool and correctly generates each subsequent number in the sequence. The reasoning is sound, as it adheres to the rules provided without fabricating results or making multiple calls in one response. However

### Create Eval summary report

In [47]:
!uv pip install pandas tabulate -q

In [48]:
import pandas as pd

In [61]:
report = pd.DataFrame(
    data={
        "result_correctness": [int(el.correct) for el in result_evals],
        "number_steps_diff": num_steps_diff,
        "trajectory_reasoning": [e.reasoning.score for e in trajectory_evals],
        "trajectory_process": [e.process.score for e in trajectory_evals],
        "trajectory_overall": [
            (e.reasoning.score + e.process.score) / 2 for e in trajectory_evals
        ],
    },
)
report.loc["Average"] = report.mean().round(2)
print(report.to_markdown())

|         |   result_correctness |   number_steps_diff |   trajectory_reasoning |   trajectory_process |   trajectory_overall |
|:--------|---------------------:|--------------------:|-----------------------:|---------------------:|---------------------:|
| 0       |                    1 |                15   |                    4   |                  4   |                  4   |
| 1       |                    1 |                10   |                    4   |                  5   |                  4.5 |
| 2       |                    1 |                 1   |                    5   |                  5   |                  5   |
| 3       |                    1 |                 4   |                    4   |                  5   |                  4.5 |
| 4       |                    1 |                 1   |                    4   |                  5   |                  4.5 |
| Average |                    1 |                 6.2 |                    4.2 |                  4.8 |

__A previous benchmark run with Qwen2.5-3b as backbone LLM__

|         |   result_correctness |   number_steps_diff |   trajectory_reasoning |   trajectory_process |   trajectory_overall |
|:--------|---------------------:|--------------------:|-----------------------:|---------------------:|---------------------:|
| 0       |                  0   |                 -10 |                    3   |                  2   |                  2.5 |
| 1       |                  0   |                  -6 |                    3   |                  4   |                  3.5 |
| 2       |                  1   |                  -1 |                    2   |                  0   |                  1   |
| 3       |                  1   |                   1 |                    4   |                  3   |                  3.5 |
| 4       |                  0   |                   1 |                    2   |                  3   |                  2.5 |
| Average |                  0.4 |                  -3 |                    2.8 |                  2.4 |                  2.6 |